In [1]:
from collections import Counter
from keras.models import load_model
import nltk
import numpy as np
import re

model = load_model('./model_0.9972.h5')

NGRAM = 5
MAXLEN = 39

alphabet = ['\x00',' '] + list('0123456789abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ')

In [2]:
def encoderData(text):
    x = np.zeros((MAXLEN, len(alphabet)))
    for i, c in enumerate(text[:MAXLEN]):
        x[i, alphabet.index(c)] = 1
    if i < MAXLEN - 1:
        for j in range(i + 1, MAXLEN):
            x[j, 0] = 1
    return x

def decoderData(x):
    x = x.argmax(axis = -1)
    return ''.join(alphabet[i] for i in x)

In [3]:
print(encoderData('Tôi yêu bạn').shape)
print(decoderData(encoderData('Tôi yêu bạn')))

(39, 198)
Tôi yêu bạn                            


In [4]:
def nltkNGRAMS(sentence, n, maxlen):
    listNGRAMS = []
    listWords = sentence.split()
    numWords = len(listWords)
    if (numWords >= n):
        for ngram in nltk.ngrams(listWords,n):
            if len(' '.join(ngram)) <= maxlen:
                listNGRAMS.append(ngram)
    else:
        listNGRAMS.append(tuple(listWords))
    return listNGRAMS

In [5]:
def guess(ngram):
    text = ' '.join(ngram)
    preds = model.predict(np.array([encoderData(text)]))

    return decoderData(preds[0]).strip('\x00')

In [6]:
guess(('ur'))

1/1 [==============================] - 1s 1s/step


'ur'

In [7]:
def addPunctation(text, correctedText):
    listPunctation = {}

    for (i,word) in enumerate(str(text).split()):
        if word[0] not in alphabet or word[-1] not in alphabet:
            startPunc = ''
            for c in word:
                if c in alphabet:
                    break
                startPunc += c

            endPunc = ''
            for c in word:
                if c in alphabet:
                    break
                endPunc += c
            endPunc = endPunc[::-1]

            listPunctation[i] = [startPunc,endPunc]

    result = ''
    for (i,word) in enumerate(str(correctedText).split()):
        if i in listPunctation:
            result += (listPunctation[i][0] + word + listPunctation[i][1]) + ' '
        else:
            result += word + ' '

    return result.strip()

In [8]:
def correctText(text):
    newText = re.sub(r'[^' + ''.join(alphabet) + ']', '', str(text))

    ngrams = list(nltkNGRAMS(newText,NGRAM,MAXLEN))
    guessedNGRAMS = list(guess(ngram) for ngram in ngrams)
    candidates = [Counter() for _ in range(len(guessedNGRAMS) + NGRAM - 1)]

    for nid, ngram in (enumerate(guessedNGRAMS)):
        for wid, word in (enumerate(re.split('\s', ngram))):
            candidates[nid+wid].update([word])

    correctedText = ' '.join(c.most_common(1)[0][0] for c in candidates if c)
    return addPunctation(text, correctedText)

In [9]:
text = input()

result = correctText(text)
print(result)

text = re.sub(r'[^' + ''.join(alphabet) + ']', '', str(text))
listText = text.split()

result = re.sub(r'[^' + ''.join(alphabet) + ']', '', str(result))
listResult = result.split()

correctWord = [(listText[i], listResult[i]) for i in range(len(listText)) if listText[i] != listResult[i]]
correctWord

1/1 [==============================] - 0s 36ms/step
Èooma


[('Moong', 'Èooma')]

In [10]:
from tkinter import *
import tkinter
from tkinter.ttk import *

window = Tk()
window.title("Spell Correction")
window.geometry("400x500")
txt = Entry(window, width=30, font=('Times New Roman',12))
txt.place(x=80,y=10)
def clicked():
    label1 = Label(window, font=('Times New Roman',12), text='Câu sau khi sửa : ' + correctText(txt.get()))
    label1.place(x=100,y=70)
button = Button(window, text='Sửa', command=clicked)
button.place(x=160,y=40)
window.mainloop()

1/1 [==============================] - 0s 38ms/step
